In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
tf.config.run_functions_eagerly(True)


print(os.getcwd())

/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model


In [2]:
data = pd.read_csv(
    "/Users/sumitsingh/Documents/GitHub/mlproject/StockAppFrontend/ml_model/source/indexProcessed.csv"
)
prices = data["Close"].values

scaler = MinMaxScaler() #normalising data into fixed min-max val
prices_scaled = scaler.fit_transform(prices.reshape(-1, 1))

print(data.head())

  Index        Date         Open         High          Low        Close  \
0   HSI  1986-12-31  2568.300049  2568.300049  2568.300049  2568.300049   
1   HSI  1987-01-02  2540.100098  2540.100098  2540.100098  2540.100098   
2   HSI  1987-01-05  2552.399902  2552.399902  2552.399902  2552.399902   
3   HSI  1987-01-06  2583.899902  2583.899902  2583.899902  2583.899902   
4   HSI  1987-01-07  2607.100098  2607.100098  2607.100098  2607.100098   

     Adj Close  Volume    CloseUSD  
0  2568.300049     0.0  333.879006  
1  2540.100098     0.0  330.213013  
2  2552.399902     0.0  331.811987  
3  2583.899902     0.0  335.906987  
4  2607.100098     0.0  338.923013  


In [3]:
# Prepare Features
window_size = 60  # Number of past days to use as input
X, y = [], []
for i in range(window_size, len(prices_scaled)):
    X.append(prices_scaled[i - window_size : i, 0])
    y.append(prices_scaled[i, 0])

X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for LSTM

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
# Build LSTM Model
model = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        tf.keras.layers.LSTM(50, return_sequences=False),
        tf.keras.layers.Dense(25),
        tf.keras.layers.Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the Model
model.save("stock_prediction_model.h5")

# Predict
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

Epoch 1/10


/Users/sumitsingh/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/sumitsingh/anaconda3/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


  73/2605 ━━━━━━━━━━━━━━━━━━━━ 7:36 180ms/step - loss: 0.0045

KeyboardInterrupt: 

In [5]:
from tensorflow.keras.models import load_model

model = load_model("stock_prediction_model.h5")

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,903 (124.62 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
model_json = model.to_json()

with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Install the required package
%pip install tensorflow

from tensorflow.keras.utils import plot_model

plot_model(
    model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True
)

Note: you may need to restart the kernel to use updated packages.
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [7]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
model.evaluate(X_test, y_test)

Epoch 1/10
  24/2605 ━━━━━━━━━━━━━━━━━━━━ 8:49 205ms/step - loss: 3.4973e-05

KeyboardInterrupt: 

In [8]:
# Install the required package
%pip install xgboost

import yfinance as yf
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

# 📌 Fetch Stock Data
def get_stock_data(stock_symbol, period="5y"):
    stock = yf.Ticker(stock_symbol)
    df = stock.history(period=period)
    df["Return"] = df["Close"].pct_change()  # Calculate daily return
    df.dropna(inplace=True)
    return df

# 📌 Feature Engineering
def add_technical_indicators(df):
    df["SMA_50"] = df["Close"].rolling(window=50).mean()
    df["SMA_200"] = df["Close"].rolling(window=200).mean()
    df["RSI"] = 100 - (100 / (1 + (df["Return"].rolling(14).mean() / df["Return"].rolling(14).std())))
    df["Target"] = (df["Close"].shift(-7) > df["Close"]).astype(int)  # Predict 7-day success
    df.dropna(inplace=True)
    return df

# 📌 Load & Process Data
stock_symbol = "RELIANCE.NS"  # Example stock
df = get_stock_data(stock_symbol)
df = add_technical_indicators(df)

# 📌 Train/Test Split
X = df[["SMA_50", "SMA_200", "RSI", "Close"]]
y = df["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Train Model
model = xgb.XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)

# 📌 Evaluate
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))

# 📌 Save Model
with open("stock_model.pkl", "wb") as f:
    pickle.dump(model, f)
    

Note: you may need to restart the kernel to use updated packages.
Model Accuracy: 0.7836538461538461


In [9]:
# Install the required package
%pip install fastapi

from fastapi import FastAPI
import yfinance as yf
import pandas as pd
import pickle

app = FastAPI()

# 📌 Load trained model
with open("stock_model.pkl", "rb") as f:
    model = pickle.load(f)


# 📌 Fetch Stock Data
def fetch_stock_data(stock_symbol):
    stock = yf.Ticker(stock_symbol)
    df = stock.history(period="1y")
    df["SMA_50"] = df["Close"].rolling(window=50).mean()
    df["SMA_200"] = df["Close"].rolling(window=200).mean()
    df["RSI"] = 100 - (
        100
        / (
            1
            + (
                df["Close"].pct_change().rolling(14).mean()
                / df["Close"].pct_change().rolling(14).std()
            )
        )
    )
    df.dropna(inplace=True)
    return df.iloc[-1][["SMA_50", "SMA_200", "RSI", "Close"]]


@app.get("/predict/{stock_symbol}")
def predict_stock(stock_symbol: str):
    try:
        features = fetch_stock_data(stock_symbol)
        prediction = model.predict([features])[0]
        success_rate = round(
            model.predict_proba([features])[0][1] * 100, 2
        )  # Probability of going up
        return {"stock": stock_symbol, "success_rate": f"{success_rate}%"}
    except Exception as e:
        return {"error": str(e)}


# Run with: uvicorn app:app --reload


Note: you may need to restart the kernel to use updated packages.


In [ ]:
async function getPrediction(stockSymbol) {
  const response = await fetch(`http://127.0.0.1:8000/predict/${stockSymbol}`);
  const data = await response.json();
  console.log(data.success_rate);  // Display predicted success rate
}